# Julia notebook for OCT and holistic regression

In [485]:
using Gurobi, JuMP
using CSV, DataFrames
using LinearAlgebra, Random, Statistics
using PyPlot

In [486]:
data = CSV.read("processed_data.csv", DataFrame)
X = Matrix(select(data, Not(["setting1", "setting2", "unitNumber", "RUL"])))
y = data[:, "RUL"];

In [487]:
X_norm = (X .- minimum(X, dims = 1)) ./ (maximum(X, dims = 1) - minimum(X, dims = 1))
y_norm = (y .- minimum(X)) ./ (maximum(y) .- minimum(y));

In [488]:
(X_train, y_train), (X_test, y_test) = IAI.split_data(:regression, X_norm, y_norm, seed = 2, train_proportion = 0.8);

In [489]:
# Create a function that transforms the matrix input matrix by adding non-linear transformations to all columns
function transform_data(X, eps = 0.1)

    n, p = size(X) # Store the matrix dimension
    # Initiate the transformed matrix with the first feature
    Xt = hcat(X[:,1], X[:,1].^2, X[:,1].^0.5, exp.(X[:,1]), log.(abs.(X[:,1]) .+ eps))

    # Loop for all other features and concatenate the transformations to the matrix
    for i = 2:p
        Xt = hcat(Xt, X[:,i], X[:,i].^2, X[:,i].^0.5, exp.(X[:,i]), log.(abs.(X[:,i]) .+ eps))
    end

    return Xt;

end;

In [490]:
features = names(select(data, Not(["setting1", "setting2", "unitNumber", "RUL"])))
features_transformed = [pre * f for f in features for pre in ["", "squared_", "root_", "exp_", "log_"]];

In [491]:
# Create a function that returns a list of feature couples whose correlation exceeds rho
function pairwise_correlation(X, rho)

    _, p = size(X) # Store the number of features
    cor_matrix = cor(X); # Compute the correlation matrix
    cor_variables = [] # Initialize an empty list to store all pairwise correlated features couples

    for i = 1:p 
        for j = 1:i-1 # Loop under all values of the lower diagonal of the correlation matrix

            if abs(cor_matrix[i,j]) > rho
                # If the correlation coefficient of features i and j exceeds rho, append the couple to the list
                push!(cor_variables, (i,j))
            end
            
        end
    end

    return cor_variables;

end;

In [538]:
function holistic_regression(X, y, rho, lambda, k, M)

    # Create the transformed matrix
    X_transformed = transform_data(X);
    n, p = size(X) # Store the input matrix size
    _, p_tilde = size(X_transformed) # Set the number of features in the transformed matrix
    augmentation = Int(p_tilde / p)
    # Compute the list of correlated feature couples
    HC = pairwise_correlation(X_transformed, rho)

    # Create JuMP model
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)

    # Introduce model variables
    @variable(model, Beta[1:p_tilde])
    @variable(model, a[1:p_tilde] >= 0)
    @variable(model, z[1:p_tilde], binary = true)

    # Robustness term constraint linearization
    @constraint(model,[j = 1:p_tilde], Beta[j] <= a[j])
    @constraint(model,[j = 1:p_tilde], -Beta[j] <= a[j])

    # Big-M integer constraint for sparsity
    @constraint(model,[j = 1:p_tilde], Beta[j] <= M*z[j])
    @constraint(model,[j = 1:p_tilde], Beta[j] >= -M*z[j])
    # Sparsity constraint
    @constraint(model, sum(z[j] for j = 1:p_tilde) <= k)

    # Non-linear transformations constraint
    @constraint(model, [j = 1:p], sum(z[augmentation*(j-1)+i] for i = 1:augmentation) <= 1)

    # Pairwise collinearity
    @constraint(model, [i = 1:length(HC)], z[HC[i][1]] + z[HC[i][2]] <= 1)

    # Implement the objective function of the problem
    @objective(model, Min, sum((y[i] - sum(X_transformed[i,j]*Beta[j] for j=1:p_tilde))^2 for i = 1:n) + lambda * sum(a[j] for j = 1:p_tilde))

    # Solve the optimization problem
    optimize!(model);

    Beta, z = JuMP.value.(Beta), Int.(JuMP.value.(z))

    y_pred = X_transformed * Beta
    r2 = 1 - sum((y_pred .- y).^2) / sum((mean(y) .- y).^2)
    print("Training R2: $(r2)")

    return Beta, z, r2

end;

In [532]:
# Specify the holisitic regression problem parameters
rho = 0.8
lambda = 1
k = 10
M = 100

R2 = []
for seed in Vector(1:10)
    (X_train, y_train), (X_test, y_test) = IAI.split_data(:regression, X_norm, y_norm, seed = seed, train_proportion = 0.8);
    # Compute the holistic regression model
    Beta, _ = holistic_regression(X_train, y_train, rho, lambda, k, M);
    X_test_transformed = transform_data(X_test);
    y_pred = Beta_0 .+ X_test_transformed * Beta;
    r2 = 1 - sum((y_pred .- y_test).^2) / sum((mean(y_train) .- y_test).^2)
    push!(R2, r2)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18


In [539]:
(X_train, y_train), (X_test, y_test) = IAI.split_data(:regression, X_norm, y_norm, seed = 1, train_proportion = 0.8);

# Compute the holistic regression model
Beta, _, _ = holistic_regression(X_train, y_train, rho, lambda, k, M);
X_test_transformed = transform_data(X_test);
y_pred = X_test_transformed * Beta;
r2 = 1 - sum((y_pred .- y_test).^2) / sum((mean(y_train) .- y_test).^2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-18
Training R2: 0.69804972093401

0.6798049184986299